In [7]:
import numpy as np
import pandas as pd
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Flatten
from keras.layers import Embedding
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, model_from_json
from keras.layers import Dense, LSTM, Dropout, Bidirectional, merge
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from IPython.display import SVG
#from keras.utils.visualize_util import model_to_dot
#import wikipedia
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool

#from mlutils import dataset
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
#import sklearn.cross_validation
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix

import os, sys, re, csv
import pandas as pd
import numpy as np
#from nltk.corpus import stopwords
from collections import Counter
from difflib import SequenceMatcher
import math
from datetime import datetime
#from nltk.tokenize import word_tokenize
from itertools import product

import time 
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool


from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis


In [8]:
#Train Data
df_train = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Microsoft/data/data.tsv', 
                       delimiter="\t",header=None)

In [9]:
#Test Data
df_test = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Microsoft/data/eval1_unlabelled.tsv',
                      delimiter="\t",header=None)

In [10]:
df_train.columns = ["id", "question", "answer", "is_ans", "seq_no"]

In [11]:
df_train.head(4)

,id,question,answer,is_ans,seq_no
0,131,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,131,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,131,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,131,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3


In [12]:
def get_cosine(str1, str2):
    
    #Counter forms a dictionary----> Frequency of words as values and key is the word
    vec1 = Counter(str1.split())
    vec2 = Counter(str2.split())
    
    #Intersection contains the set of common words
    intersection = set(vec1.keys()) & set(vec2.keys())
    
    #W1W2W2=[1,2,3] and W1W2W3=[2,4,6] ,it finds the a.b value
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    #|a| and |b| are computed through the process
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    #cosine value is computed
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [13]:
def DistJaccard(str1, str2):
    str1 = set(str1.split())
    str2 = set(str2.split())
    #a_intersection_b ---unique
    numerator = len(str1 & str2)
    #a_union_b---unique
    denominator = len(str1 | str2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


In [14]:
def shared_2gram(q1, q2):
    #zip will combine the first doc with the second doc element by element
    q1_2gram = set([i for i in zip(q1.split(), q1.split()[1:])])
    q2_2gram = set([i for i in zip(q2.split(), q2.split()[1:])])
    shared_2gram = q1_2gram.intersection(q2_2gram)
    if len(q1_2gram)==0 or len(q2_2gram) == 0:
        return 0
    return len(shared_2gram)*2.0/(len(q1_2gram) + len(q2_2gram))

In [15]:
def shared_3gram(q1, q2):
    q1_3gram = set([i for i in zip(q1.split(), q1.split()[1:], q1.split()[2:])])
    q2_3gram = set([i for i in zip(q2.split(), q2.split()[1:], q2.split()[2:])])
    shared_3gram = q1_3gram.intersection(q2_3gram)
    if len(q1_3gram)==0 or len(q2_3gram) == 0:
        return 0
    return len(shared_3gram)*1.0/(len(q1_3gram) + len(q2_3gram))


In [ ]:
def worker(data):
    cosine_sim, jaccard_sim, shared_2gram_sim, shared_3gram_sim = np.zeros((len(data), )), np.zeros((len(data), )), np.zeros((len(data), )), np.zeros((len(data), ))
    #print '*'*50
    for i in range(0,len(data)):
        d=data.iloc[i]
        cosine_sim[i] =   get_cosine(str(d[1]), str(d[2]))
        #abb_desc_sim[i] = get_desc_sim(str(d[1]), str(d[2]))
        jaccard_sim[i] =  DistJaccard(str(d[1]), str(d[2]))
        shared_2gram_sim[i] = shared_2gram(str(d[1]), str(d[2]))
        shared_3gram_sim[i] = shared_3gram(str(d[1]), str(d[2]))
    result = pd.DataFrame()
    result['cosine'] = pd.Series(list(cosine_sim))
    result['jaccard'] = pd.Series(list(jaccard_sim))
    result['shared_2gram'] = pd.Series(list(shared_2gram_sim))
    result['shared_3gram'] = pd.Series(list(shared_3gram_sim))
    
    
    
    
    return result

In [ ]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
import time
t1=time.time()
num_partitions = 16
n_cores=8
dist_df = parallelize_dataframe(df_train, worker)
t2=time.time()

print t2-t1

In [ ]:
x_feat=dist_df
y=df_train['is_ans']

In [ ]:
x_feat.columns

In [ ]:
plt.hist(x_feat[y==0]['cosine'])

In [ ]:
plt.hist(x_feat[y==1]['cosine'])

In [ ]:
plt.hist(x_feat[y==0]['jaccard'])

In [ ]:
plt.hist(x_feat[y==1]['jaccard'])

In [ ]:
plt.hist(x_feat[y==0]['shared_2gram'])

In [ ]:
plt.hist(x_feat[y==1]['shared_2gram'])

In [ ]:
plt.hist(x_feat[y==0]['shared_3gram'])

In [ ]:
plt.hist(x_feat[y==1]['shared_3gram'])

In [29]:
#Adding the Sentiment part

In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [23]:
def print_sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(snt)))

In [40]:
print_sentiment_scores('The food is bad')

The food is bad------------------------- {'neg': 0.538, 'neu': 0.462, 'pos': 0.0, 'compound': -0.5423}


In [ ]:
params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "num_leaves" : 10,
        "min_child_samples" : 100,
        "n_estimators":1000,
        "learning_rate" : 0.03,
        "bagging_fraction" : 0.7,
        "feature_fraction" : 0.5,
        "bagging_frequency" : 5,
        "bagging_seed" : 2018,
        "subsample" : .9,
        "colsample_bytree": .9,
        "verbosity" : -1,
    }

In [ ]:
import lightgbm as lgb

In [ ]:
lgtrain = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(params, lgtrain,1000)

In [ ]:
y_pred_prob=lgb_model.predict(x_test)

In [ ]:
#y_pred

In [ ]:
np.true_divide(len(x_train[y==1]),len(x_train[y==0]))

In [ ]:
y_pred_prob[0:10]

In [ ]:
#y_pred_prob[0:9]

TEST PREDICTION

In [ ]:
df_test.columns = ["id", "question", "answer", "seq_no"]

In [ ]:
df_test.head()

In [ ]:
import time
t1=time.time()
num_partitions = 16
n_cores=7
dist_df_test = parallelize_dataframe(df_test, worker)
t2=time.time()

print t2-t1

In [ ]:
predictions=lgb_model.predict(dist_df_test)

In [ ]:
all_scores = {}
i=-1
for qid, pred in zip(df_test['id'], predictions):
        i=i+1
        all_scores[qid] = predictions[0+i:10+i]

In [ ]:
#len(all_scores.keys())
all_scores = {}
 
for qid, pred in zip(df_test['id'], predictions):
   
    if qid in all_scores:
        all_scores[qid].append(pred)
    else:
        all_scores[qid] = [pred]
 

In [ ]:
submissionFileName = "answer2.tsv"
fw = open(submissionFileName, "w")
 
for query_id in all_scores:
    scores = all_scores[query_id]
    scores_str = [str(sc) for sc in scores] # convert all scores to string values
    scores_str = "\t".join(scores_str) # join all scores in list to make it one string with  tab delimiter. 
    fw.write(str(query_id)+"\t"+scores_str+"\n")
fw.close()

In [ ]:
#all_scores

In [267]:
from lib.google_search_results import GoogleSearchResults
query = GoogleSearchResults({"q":"what is a corporate"})
json_results = query.get_json()

In [266]:
#json_results
# a=json_results['organic_results'][0]['snippet']
# a

In [260]:
pd.options.display.max_rows = 4000


In [269]:
#json_results['knowledge_graph']['description']

In [241]:
a=json_results['organic_results'][0]['snippet']
a

u'The symptoms are similar but the mouse will be in much worse condition: runny eyes. sneezing. wheezing. shaking. fluctuating body temperature. tiredness. loss of appetite. dull coat.'

In [242]:
b=json_results['organic_results'][1]['snippet']

In [249]:
a+b

u"The symptoms are similar but the mouse will be in much worse condition: runny eyes. sneezing. wheezing. shaking. fluctuating body temperature. tiredness. loss of appetite. dull coat.Mice can become unwell and go downhill quickly, but may often only show subtle signs of being in pain/distress/suffering, until it's very severe. Pain/distress/suffering can usually be indicated by behavioural changes. Taking to a vet immediately if showing signs of illness."

In [1]:
df_train.iloc[15]['answer']

NameError: name 'df_train' is not defined

In [ ]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_feat, y, test_size=0.05, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(random_state=0)
log_model.fit(x_train,y_train)

In [ ]:
log_model.score(x_test,y_test)

In [ ]:
#print (recall_score(y_test, log_model.predict(x_test)))
#print (precision_score(y_test, log_model.predict(x_test)))

In [ ]:
y_pred=log_model.predict(x_test)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
len(y_test)